# Initial Setup

In [ ]:
#@title Setting up the notebook

### Installing dependencies
!pip install openai
!pip install anthropic
!apt-get update
!apt-get install -y iverilog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 13.6 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://cli.github.com/packages stable InRelease [3,917 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,611 kB]
Get:

In [ ]:
#@title Select Model
#define the model to be used
model_choice = "gpt-5.2"
#model_choice = "gpt-4o"
#model_choice = "claude-sonnet-4-5"
#model_choice = "gemini-2.5-flash-preview-04-17"
#model_choice = "gemini-2.5-flash"

In [ ]:
#@title Utility functions

import sys
import os
import openai
import anthropic
import google.genai.errors
from google import genai
from google.genai import types
from abc import ABC, abstractmethod
import re






################################################################################
### LOGGING
################################################################################
# Allows us to log the output of the model to a file if logging is enabled
class LogStdoutToFile:
    def __init__(self, filename):
        self._filename = filename
        self._original_stdout = sys.stdout

    def __enter__(self):
        if self._filename:
            sys.stdout = open(self._filename, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self._filename:
            sys.stdout.close()
        sys.stdout = self._original_stdout


################################################################################
### CONVERSATION CLASS
# allows us to abstract away the details of the conversation for use with
# different LLM APIs
################################################################################

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file

        if self.log_file and os.path.exists(self.log_file):
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        """Add a new message to the conversation."""
        self.messages.append({'role': role, 'content': content})

        if self.log_file:
            with open(self.log_file, 'a') as file:
                file.write(f"{role}: {content}\n")

    def get_messages(self):
        """Retrieve the entire conversation."""
        return self.messages

    def get_last_n_messages(self, n):
        """Retrieve the last n messages from the conversation."""
        return self.messages[-n:]

    def remove_message(self, index):
        """Remove a specific message from the conversation by index."""
        if index < len(self.messages):
            del self.messages[index]

    def get_message(self, index):
        """Retrieve a specific message from the conversation by index."""
        return self.messages[index] if index < len(self.messages) else None

    def clear_messages(self):
        """Clear all messages from the conversation."""
        self.messages = []

    def __str__(self):
        """Return the conversation in a string format."""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])

################################################################################
### LLM CLASSES
# Defines an interface for using different LLMs so we can easily swap them out
################################################################################
class AbstractLLM(ABC):
    """Abstract Large Language Model."""

    def __init__(self):
        pass

    @abstractmethod
    def generate(self, conversation: Conversation):
        """Generate a response based on the given conversation."""
        pass


class ChatGPT(AbstractLLM):
    """ChatGPT Large Language Model."""

    def __init__(self, model_id=model_choice):
        super().__init__()
        openai.api_key=os.environ['OPENAI_API_KEY']
        self.client = openai.OpenAI()
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_choices=1):
        messages = [{"role" : "user", "content" : msg["content"]} for msg in conversation.get_messages()]

        response = self.client.chat.completions.create(
            model=self.model_id,
            messages = messages,
        )

        return response.choices[0].message.content
class Claude(AbstractLLM):
      def __init__(self, model_id=model_choice):
        super().__init__()
        self.client = anthropic.Anthropic(api_key=os.environ['CLAUDE_API_KEY'])
        self.model_id = model_id

      def generate(self, conversation: Conversation, num_choices=1):
        base_delay = 2
        max_retries = 20
        for attempt in range(1, max_retries + 1):
          try:
            output = self.client.messages.create(
                      model=model_choice,
                      max_tokens=16384,
                      messages=[{"role" : msg["role"], "content" : msg["content"]} for msg in conversation.get_messages()]
                  ).content[0].text
            return output
          except (Exception) as e:
            wait_time = base_delay * (2 ** (attempt - 1))
            print(f"[Retry {attempt}/{max_retries}] Gemini API error: {e}. Retrying in {wait_time:.1f} seconds...")
            time.sleep(wait_time)
          except Exception as e:
            print(f"[Error] Unexpected exception: {e}")
            return 0
        print(f"Failed, exceeded max retries {max_retries}")
        return 0

class Gemini(AbstractLLM):
      def __init__(self, model_id=model_choice):
        super().__init__()
        self.gemini_client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])
        self.model_id = model_id

      def generate(self, conversation: Conversation, num_choices=1):

          output = self.gemini_client.models.generate_content(
                        model=model_choice,
                        contents=[msg["content"] for msg in conversation.get_messages()],
                        config=types.GenerateContentConfig(
                            max_output_tokens=16384,
                            temperature=0.6,
                            topP=0.95,
                        )
                    ).text
          return output
################################################################################
### PARSING AND TEXT MANIPULATION FUNCTIONS
################################################################################
def find_verilog_modules(markdown_string, module_name='top_module'):

    module_pattern1 = r'\bmodule\b\s+\w+\s*\([^)]*\)\s*;.*?endmodule\b'

    module_pattern2 = r'\bmodule\b\s+\w+\s*#\s*\([^)]*\)\s*\([^)]*\)\s*;.*?endmodule\b'

    module_matches1 = re.findall(module_pattern1, markdown_string, re.DOTALL)

    module_matches2 = re.findall(module_pattern2, markdown_string, re.DOTALL)

    module_matches = module_matches1 + module_matches2

    if not module_matches:
        return []

    return module_matches

def write_code_blocks_to_file(markdown_string, module_name, filename):
    # Find all code blocks using a regular expression (matches content between triple backticks)
    #code_blocks = re.findall(r'```(?:\w*\n)?(.*?)```', markdown_string, re.DOTALL)
    code_match = find_verilog_modules(markdown_string, module_name)

    if not code_match:
        print("No code blocks found in response")
        exit(3)

    # Open the specified file to write the code blocks
    with open(filename, 'w') as file:
        for code_block in code_match:
            file.write(code_block)
            file.write('\n')

def generate_verilog(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT()
    elif model_type == "Claude":
      model = Claude()
    elif model_type == "Gemini":
      model = Gemini()
    else:
        raise ValueError("Invalid model type")
    return(model.generate(conv))


In [ ]:
#@title Feedback Loop
import subprocess
import sys
import os
import time
import numpy as np


def _read_files(file_list):
    """Concatenate a list of text files in order. Missing/None entries are ignored."""
    if not file_list:
        return ""
    buf = []
    for p in file_list:
        if not p:
            continue
        with open(p, "r") as f:
            buf.append(f.read())
            if not buf[-1].endswith("\n"):
                buf.append("\n")
    return "".join(buf)


def _truncate_text(s, max_chars=12000):
    """Keep prompts bounded; preserve the *end* of long logs which usually has the key error lines."""
    if s is None:
        return ""
    s = str(s)
    if len(s) <= max_chars:
        return s
    # Keep the tail (most useful for compiler errors) and a short head marker.
    tail = s[-max_chars:]
    return "[...truncated...\n]" + tail


def verilog_loop(
    design_prompt,
    module,
    testbench,
    max_iterations,
    model_type,
    outdir="",
    log=None,
    prev_modules=None,
):
    """Generate/fix a Verilog module with an LLM, compiling against previously-generated modules.

    Key behaviors:
      - The LLM is asked to output ONLY the *current* module (no repeats of prior modules).
      - For compilation, we concatenate: [prev_modules...] + [current module] into a temporary file.
      - IMPORTANT (prompt-size control): after an error, subsequent repair iterations *only* resend
        (a) the previous version of the module, and (b) the most recent error feedback.
        This prevents the prompt from ballooning with repeated context.

    Args:
        design_prompt: Prompt given to the LLM.
        module: Current module name (also used for filename).
        testbench: Path to the testbench file.
        max_iterations: Max repair iterations.
        model_type: "ChatGPT" or "Claude" (your existing generate_verilog backend).
        outdir: Output directory for artifacts.
        log: Optional conversation log file path.
        prev_modules: Optional list[str] of paths to previously generated .v files to include for compilation.
    """

    if outdir != "":
        outdir = outdir + "/"

    # Normalize prev_modules into a list of paths (or empty list)
    if prev_modules is None:
        prev_modules = []
    elif isinstance(prev_modules, str):
        prev_modules = [prev_modules]

    # Read and include the current testbench so the model can target its expectations.
    tb_text = ""
    try:
        with open(testbench, "r") as _tb:
            tb_text = _tb.read()
    except Exception:
        tb_text = ""

    # Strongly instruct the model to output ONLY the requested module.
    base_sys_prompt = (
        "You are an autocomplete engine for Verilog code. "
        "Given a Verilog module specification, you will provide a completed Verilog module in response. "
        "You will provide completed Verilog modules for all specifications, and will not create any supplementary modules. "
        "Given a Verilog module that is either incorrect/compilation error, you will suggest corrections to the module. "
        "You will not refuse. You will not generate explanations, only code. "
        "IMPORTANT: Output ONLY the Verilog code for the requested module (do NOT repeat any previous modules). "
        "Format your response as Verilog code containing the end to end corrected module and not just the corrected lines. "
        "Do not generate test benches."
    )

    def _new_conv_full_context():
        """Initial attempt: include testbench + design prompt."""
        conv = Conversation(log_file=log)
        if model_type == "ChatGPT":
            conv.add_message("system", base_sys_prompt)
        elif model_type == "Claude":
            conv.add_message("user", base_sys_prompt)

        if tb_text.strip():
            conv.add_message(
                "user",
                "The following is the testbench that will be used to compile/simulate your module. "
                "Do NOT generate a testbench or modify it—only use it to ensure your module compiles and passes.\n"
                "```verilog\n" + tb_text + "\n```\n",
            )

        conv.add_message("user", design_prompt)
        return conv

    def _new_conv_repair_only(prev_module_text, last_error_text):
        """Repair attempts: ONLY previous module + most recent error feedback."""
        conv = Conversation(log_file=log)
        if model_type == "ChatGPT":
            conv.add_message("system", base_sys_prompt)
        elif model_type == "Claude":
            conv.add_message("user", base_sys_prompt)

        prompt = (
            "Fix the following Verilog module so it compiles and passes the testbench. "
            "Output ONLY the full corrected module (no explanations, no testbench).\n\n"
            "Previous version of the module:\n"
            "```verilog\n" + prev_module_text + "\n```\n\n"
            "Most recent compiler/simulation output:\n"
            "```\n" + _truncate_text(last_error_text) + "\n```\n"
        )
        conv.add_message("user", prompt)
        return conv

    # Start with full context for the first generation.
    conv = _new_conv_full_context()

    success = False
    timeout = False

    iterations = 0
    timelist_total = []
    timelist_gen = []
    timelist_error = []

    # Persisted (single-module) file
    module_file = os.path.join(outdir, module + ".v")
    # Temporary concatenated file used only for compilation
    combined_file = os.path.join(outdir, module + "__combined.v")

    status = ""
    last_module_text = ""
    last_error_text = ""

    while not (success or timeout):
        start_total = time.time()

        # Generate (or fix) ONLY the current module.
        response = generate_verilog(conv, model_type)
        end_gen = time.time()
        start_error = time.time()

        # Keep the conversation clean: assistant message is only the current module text.
        conv.add_message("assistant", response)

        # Write just the current module to its own file.
        write_code_blocks_to_file(response, module, module_file)

        # Build concatenated source for compilation: prior modules + current module
        prev_text = _read_files(prev_modules)
        with open(module_file, "r") as f:
            curr_text = f.read()
        last_module_text = curr_text

        with open(combined_file, "w") as f:
            f.write(prev_text)
            if prev_text and not prev_text.endswith("\n"):
                f.write("\n")
            f.write(curr_text)

        # Compile using the combined file (so the hierarchy is visible to iVerilog).
        proc = subprocess.run(
            ["iverilog", "-o", os.path.join(outdir, module), combined_file, testbench],
            capture_output=True,
            text=True,
        )

        success = False

        if proc.returncode != 0:
            status = "Error compiling testbench"
            print(status)
            last_error_text = (
                "iverilog failed. stderr:\n" + (proc.stderr or "") + "\nstdout:\n" + (proc.stdout or "")
            )
        elif proc.stderr != "":
            status = "Warnings compiling testbench"
            print(status)
            last_error_text = (
                "iverilog produced warnings. stderr:\n" + (proc.stderr or "") + "\nstdout:\n" + (proc.stdout or "")
            )
        else:
            proc = subprocess.run(["vvp", os.path.join(outdir, module)], capture_output=True, text=True)
            result = proc.stdout
            if "passed!" not in result:
                status = "Error running testbench"
                print(status)
                last_error_text = "Simulation output:\n" + (proc.stdout or "") + "\n" + (proc.stderr or "")
            else:
                status = "Testbench ran successfully"
                print(status)
                last_error_text = ""
                success = True

        # Write iteration log (bounded by the conversation size, which is kept small on repair)
        with open(os.path.join(outdir, "log_iter_" + str(iterations) + ".txt"), "w") as file:
            file.write("\n".join(str(i) for i in conv.get_messages()))
            file.write("\n\n Iteration status: " + status + "\n")

        if not success:
            # On failure: DO NOT keep adding more and more context.
            # Rebuild the conversation to contain ONLY the previous module + last error feedback.
            conv = _new_conv_repair_only(last_module_text, last_error_text)

        if iterations >= max_iterations:
            timeout = True

        iterations += 1
        end_time = time.time()
        timelist_gen.append(end_gen - start_total)
        timelist_error.append(end_time - start_error)
        timelist_total.append(end_time - start_total)

    print("Total time: ", np.sum(timelist_total))
    print("Generation time: ", np.sum(timelist_gen))
    print("Error handling time: ", np.sum(timelist_error))
    return (np.sum(timelist_total), np.sum(timelist_gen), np.sum(timelist_error))


In [ ]:
#@title Hierarchical Loop
def hier_gen(submods, max_iterations=10):
  """Hierarchically generate a design by building submodules in order.

  submods: list of tuples/lists like:
      [ (file_basename, human_name, io_string), ... ]
    Example:
      [ ("mux2_1", "2-to-1 mux", "input wire ..."), ("mux4to1", ...), ... ]

  This version:
    - Shows previously-generated modules to the LLM for context (so it can instantiate them),
      but explicitly instructs it to output ONLY the *current* module.
    - Passes a growing list of prior .v files to verilog_loop, which concatenates them ONLY
      for iVerilog compilation (not in the LLM response).
  """
  totaltime = []
  gentime = []
  errortime = []

  done = ""
  prev_verilog_files = []  # paths to previously generated .v files (for compilation only)

  for i in range(len(submods)):
    curr = submods[i][1]
    fcurr = submods[i][0]
    iocurr = submods[i][2]
    overall = submods[-1][1]

    if not os.path.isdir(fcurr):
      os.mkdir(fcurr)

    # Build prompt:
    # - Include all previous modules (as context) but do NOT request them to be repeated.
    if i == 0:
      prompt = (
        "// We will be generating a " + overall + " hierarchically in Verilog.\n"
        "// IMPORTANT: Output ONLY the module requested below. Do NOT repeat any previous modules.\n"
        "// Begin by generating: " + curr + "\n"
        "module " + fcurr + "(" + iocurr + ")\n"
        "// Insert code here\n"
        "endmodule\n"
      )
    else:
      # Concatenate prior generated modules to give the model correct interface/context.
      prev_text = ""
      for p in prev_verilog_files:
        with open(p, "r") as f:
          prev_text += f.read()
          if not prev_text.endswith("\n"):
            prev_text += "\n"

      prompt = (
        "// We are generating a " + overall + " hierarchically in Verilog.\n"
        "// Previously generated modules (for reference only):\n"
        + prev_text +
        "\n// IMPORTANT: Do NOT repeat any previous modules in your response.\n"
        "// Output ONLY the new module requested below, using the previous modules hierarchically.\n"
        "// Now generate: " + curr + "\n"
        "module " + fcurr + "(" + iocurr + ")\n"
        "// Insert code here\n"
        "endmodule\n"
      )

    module = fcurr
    testbench = "./" + fcurr + "tb.v"
    model = os.environ["MODEL"]
    outdir = "./" + fcurr
    log = "./" + fcurr + "/log.txt"

    # Compile with all previous modules (concatenated inside verilog_loop), but only generate current module.
    total, gen, error = verilog_loop(prompt, module, testbench, max_iterations, model, outdir, log, prev_modules=prev_verilog_files)

    totaltime.append(total)
    gentime.append(gen)
    errortime.append(error)

    # Add the current module file to the compilation context for subsequent modules.
    prev_verilog_files.append(os.path.join(outdir, module + ".v"))
    done = done + curr + ", "

  print("Overall Total time: ", np.sum(totaltime))
  print("Overall Generation Time: ", np.sum(gentime))
  print("Overall Error handling time: ", np.sum(errortime))


# Setting the API Key

In [ ]:
### OpenAI API KEY

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

#Please insert your own GPT-4 enabled API key as a string here:
os.environ["OPENAI_API_KEY"] = "API KEY HERE"
#os.environ['CLAUDE_API_KEY'] = "API KEY HERE"
#os.environ['GEMINI_API_KEY'] ="API KEY HERE"
os.environ["MODEL"] = "ChatGPT"
#os.environ["MODEL"] = "Claude"
#os.environ["MODEL"] = "Gemini"

#Mux Hierarchy Example

In [ ]:
#@title Pull Testbenches

#From the ROME GitHub Repo

!git clone https://github.com/ajn313/ROME-LLM
!cp ./ROME-LLM/testbenches/mux/mux2_1tb.v ./mux2_1tb.v
!cp ./ROME-LLM/testbenches/mux/mux4_1tb.v ./mux4_1tb.v
!cp ./ROME-LLM/testbenches/mux/mux8_1tb.v ./mux8_1tb.v
!cp ./ROME-LLM/testbenches/mux/mux16_1tb.v ./mux16_1tb.v
!cp ./ROME-LLM/testbenches/mux/mux32_1tb.v ./mux32_1tb.v
!cp ./ROME-LLM/testbenches/mux/mux64_1tb.v ./mux64_1tb.v

fatal: destination path 'ROME-LLM' already exists and is not an empty directory.


In [ ]:
#@title Submodules


### Each step is structured as ["filename","natural language description","interface"]
submodules = [
    ["mux2_1","2-to-1 multiplexer","input wire in1, input wire in2, input wire select, output wire out"],
    ["mux4_1","4-to-1 multiplexer","input [1:0] sel, input [3:0] in, output reg out"],
    ["mux8_1","8-to-1 multiplexer","input [2:0] sel, input [7:0] in, output reg out"],
    ["mux16_1","16-to-1 multiplexer","input [3:0] sel, input [15:0] in, output reg out"],
    ["mux32_1","32-to-1 multiplexer","input [4:0] sel, input [31:0] in, output reg out"],
    ["mux64_1","64-to-1 multiplexer","input [5:0] sel, input [63:0] in, output reg out"],
]

In [ ]:
hier_gen(submodules)

Testbench ran successfully
Total time:  1.2462611198425293
Generation time:  1.2330260276794434
Error handling time:  0.013234853744506836
Testbench ran successfully
Total time:  2.566303014755249
Generation time:  2.5523438453674316
Error handling time:  0.013958930969238281
Testbench ran successfully
Total time:  1.8827190399169922
Generation time:  1.8670587539672852
Error handling time:  0.01566004753112793
Testbench ran successfully
Total time:  1.9576399326324463
Generation time:  1.9428246021270752
Error handling time:  0.014814615249633789
Testbench ran successfully
Total time:  2.2036914825439453
Generation time:  2.1851842403411865
Error handling time:  0.01850724220275879
Testbench ran successfully
Total time:  2.21122407913208
Generation time:  2.1887948513031006
Error handling time:  0.02242898941040039
Overall Total time:  12.067838668823242
Overall Generation Time:  11.969232320785522
Overall Error handling time:  0.09860467910766602


# Decoder example

In [ ]:
#@title Pull Testbenches

#From the ROME GitHub Repo

!git clone https://github.com/ajn313/ROME-LLM
!cp ./ROME-LLM/testbenches/decoder/decoder_2_to_4tb.v ./decoder_2_to_4tb.v
!cp ./ROME-LLM/testbenches/decoder/decoder_3_to_8tb.v ./decoder_3_to_8tb.v
!cp ./ROME-LLM/testbenches/decoder/decoder_5_to_32tb.v ./decoder_5_to_32tb.v

fatal: destination path 'ROME-LLM' already exists and is not an empty directory.


In [ ]:
#@title Submodules


### Each step is structured as ["filename","natural language description","interface"]
submodules = [
    ["decoder_2_to_4","2-to-4 decoder","input [1:0] in, output reg [3:0] out"],
    ["decoder_3_to_8","3-to-8 decoder","input [2:0] in, output reg [7:0] out"],
    ["decoder_5_to_32","5-to-32 decoder","input [4:0] in, output reg [31:0] out"],
]

In [ ]:
hier_gen(submodules)

Testbench ran successfully
Total time:  2.037503719329834
Generation time:  1.9895753860473633
Error handling time:  0.0479280948638916
Testbench ran successfully
Total time:  2.487109661102295
Generation time:  2.471979856491089
Error handling time:  0.015129566192626953
Testbench ran successfully
Total time:  2.6478347778320312
Generation time:  2.630598306655884
Error handling time:  0.01723623275756836
Overall Total time:  7.17244815826416
Overall Generation Time:  7.092153549194336
Overall Error handling time:  0.08029389381408691


# AES Example

In [ ]:
#@title Pull Testbenches

#From the ROME GitHub Repo

!git clone https://github.com/ajn313/ROME-LLM
!cp ./ROME-LLM/testbenches/aes/aes_coretb.v ./aes_coretb.v
!cp ./ROME-LLM/testbenches/aes/aes_decipher_blocktb.v ./aes_decipher_blocktb.v
!cp ./ROME-LLM/testbenches/aes/aes_encipher_blocktb.v ./aes_encipher_blocktb.v
!cp ./ROME-LLM/testbenches/aes/aes_key_memtb.v ./aes_key_memtb.v
!cp ./ROME-LLM/testbenches/aes/aes_sboxtb.v ./aes_sboxtb.v
!cp ./ROME-LLM/testbenches/aes/aes_inv_sboxtb.v ./aes_inv_sboxtb.v
!cp ./ROME-LLM/testbenches/aes/aestb.v ./aestb.v

Cloning into 'ROME-LLM'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 210 (delta 111), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (210/210), 986.73 KiB | 16.45 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [ ]:
#@title Submodules


### Each step is structured as ["filename","natural language description","interface"]
submodules = [
    ["aes_sbox","AES sbox","input wire [31 : 0]  sboxw, output wire [31 : 0] new_sboxw"],
    ["aes_inv_sbox","AES inverse sbox","input wire [31 : 0]  sboxw, output wire [31 : 0] new_sboxw"],
    ["aes_key_mem","AES key memory","input wire clk, input wire reset_n, input wire [255 : 0] key, input wire keylen, input wire init, input wire [3 : 0] round,  output wire [127 : 0] round_key, output wire ready, output wire [31 : 0] sboxw, input wire [31 : 0] new_sboxw"],
    #["aes_encipher_block","AES encipher block","input wire clk, input wire reset_n, input wire next, input wire keylen, output wire [3 : 0] round, input wire [127 : 0] round_key, output wire [31 : 0] sboxw, input wire [31 : 0] new_sboxw, input wire [127 : 0] block, output wire [127 : 0] new_block, output wire ready"],
    #["aes_decipher_block","AES decipher block","input wire clk, input wire reset_n, input wire next, input wire keylen, output wire [3 : 0] round, input wire [127 : 0] round_key, input wire [127 : 0] block, output wire [127 : 0] new_block, output wire ready"],
    #["aes_core","AES core","input wire clk, input wire reset_n, input wire encdec,input wire init, input wire next, output wire ready, input wire [255 : 0] key, input wire keylen, input wire [127 : 0] block, output wire [127 : 0] result, output wire result_valid"],
    #["aes","AES block cipher","input wire clk, input wire reset_n, input wire cs, input wire we, input wire [7 : 0] address, input wire [31 : 0] write_data, output wire [31 : 0] read_data"],
]

In [ ]:
hier_gen(submodules)

Testbench ran successfully
Total time:  44.62957048416138
Generation time:  44.59063363075256
Error handling time:  0.038936614990234375
Testbench ran successfully
Total time:  55.17902970314026
Generation time:  52.81671118736267
Error handling time:  2.3623180389404297
Error compiling testbench
Error compiling testbench
Warnings compiling testbench
Error compiling testbench
Warnings compiling testbench
Warnings compiling testbench
Error running testbench
Testbench ran successfully
Total time:  258.8076238632202
Generation time:  258.60989356040955
Error handling time:  0.19772672653198242
Error compiling testbench


KeyboardInterrupt: 